In [55]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from collections import Counter
from time import sleep
from sklearn.metrics import accuracy_score
session = requests.session()

#### Функция ниже получает код страницы для поиска тегов

In [6]:
def get_soup(link_part):
    ua = UserAgent(verify_ssl=False)
    headers = {'User-Agent': ua.random}
    link = 'https://irecommend.ru' + link_part
    response = session.get(link, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    return soup

#### Функция для выбора слов и добавления их в словарь
Берутся тексты длиной более 200 слов. 

Сначала я составляла обычные каунтеры, но тогда результат получался плохой - получалось очень много названий конкретных фильмов и имен. Я связываю это с тем, что некоторые отзывы были значительно больше остальных, поэтому слова из этих отзывов значительно перевешивали. Чтобы попробовать от этого избавиться, я считаю частотность по тексту (кол-во этот слова/кол-во всех слов в тексте) и ее прибавляю к текущему значению в словаре. 

In [4]:
def get_text_lemmas(text, freq, count):
    counter = Counter()
    text = text.text
    words = nltk.word_tokenize(text.lower())
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    if len(lemmas) > 200:
        count += 1
        for lemma in lemmas:
            if lemma.isalpha():
                counter[lemma] += 1
    for word in dict(counter).keys():
        freq[word] += counter[word]/len(lemmas)
    return freq, count

#### Функция собирает не менее 30 отзывов и собирает словарь
По каждому фильму не больше 2 плохих и 2 хороших отзывов, чтобы не было перевеса слов, относящихся к определенному жанру или фильму.

In [30]:
def find_words(link):
    b_count_all = 0
    g_count_all = 0
    b_freq = Counter()
    g_freq = Counter()
    
    soup = get_soup(link)
    all_films = soup.find_all('a', {'class': 'read-all-reviews-link-bottom read-all-reviews-link'})
    
    for film in all_films:
        b_count = 0
        g_count = 0
        if b_count_all > 30 and g_count_all > 30:
            break
        sleep(10)
        soup = get_soup(film.get('href'))
        all_reviews = soup.find_all('a', {'class': 'more'})
        
        for review in all_reviews:
            sleep(10)
            soup = get_soup(review.get('href'))
            recommend = soup.find('span', {'class': 'verdict'})
            
            if recommend.text == 'не рекомендует' and b_count <= 2 and b_count_all <= 30:
                text = soup.find('div', {'class': 'description hasinlineimage'})
                b_freq, b_count = get_text_lemmas(text, b_freq, b_count)
                b_count_all += b_count
                
            elif g_count <= 2 and g_count_all <= 30:
                text = soup.find('div', {'class': 'description hasinlineimage'})
                g_freq, g_count = get_text_lemmas(text, g_freq, g_count)
                g_count_all += g_count
                
            elif g_count > 2 and b_count > 2:
                break
                
    return b_freq, g_freq, b_count_all, g_count_all

#### Собираем словари и смотрим содержимое
Было собрано по 33 отзыва каждой категории. Очень много частотных слов, списки ни о чем не говорят.

In [31]:
link = '/category/filmy'
bad_freq, good_freq, b_count_all, g_count_all = find_words(link)

In [59]:
b_count_all, g_count_all

(33, 33)

In [35]:
bad_freq.most_common(15)

[('и', 0.5378129689888559),
 ('в', 0.39058127929404646),
 ('не', 0.3888307507659),
 ('фильм', 0.30171875052126335),
 ('на', 0.19723604679076498),
 ('что', 0.17145891701787222),
 ('но', 0.1691846920627757),
 ('это', 0.16692882453572386),
 ('он', 0.16436441215696262),
 ('я', 0.16368935203559262),
 ('весь', 0.14629756267375094),
 ('с', 0.1382083033589317),
 ('быть', 0.11713984116980514),
 ('как', 0.08697329060788114),
 ('о', 0.08190591509647052)]

In [36]:
good_freq.most_common(15)

[('и', 0.5494357201808143),
 ('в', 0.4011236700064026),
 ('фильм', 0.24964370411324904),
 ('не', 0.24165901205663803),
 ('на', 0.18510632686128478),
 ('он', 0.1674328990918463),
 ('что', 0.1607037606587872),
 ('я', 0.1459989414092995),
 ('с', 0.13090616239170158),
 ('который', 0.12622684384467367),
 ('но', 0.12062355645646797),
 ('это', 0.11292778782128195),
 ('весь', 0.1110795363449208),
 ('очень', 0.08910775754872191),
 ('этот', 0.08854895542621703)]

#### Функция удаляет стоп-слова
Я не стала отбирать слова по частотности, потому что не ясно, где устанавливать нижнюю границу.

In [37]:
def select_words(freq):
    stopWords = set(stopwords.words('russian'))
    selected_freq = Counter()
    for word in dict(freq).keys():
        if word not in stopWords:
            selected_freq[word] = freq[word]
    return selected_freq

#### Функция удаляет общие для двух словарей слова

In [38]:
def delete_common_words(good_freq, bad_freq):
    good = dict(good_freq)
    bad = dict(bad_freq)
    keys = list(bad.keys())
    for b_key in keys:
        if b_key in good.keys():
            del bad[b_key]
            del good[b_key]
    return Counter(good), Counter(bad)

In [39]:
new_bad_freq = select_words(bad_freq)
new_good_freq = select_words(good_freq)
pos_freq, neg_freq = delete_common_words(new_good_freq, new_bad_freq)

In [40]:
pos_freq.most_common(15)

[('дельфин', 0.025),
 ('заповедник', 0.016734105610770003),
 ('дочь', 0.015946329205949163),
 ('природа', 0.015425416693840484),
 ('произведение', 0.014968827018486218),
 ('соната', 0.014877680121966736),
 ('композитор', 0.013701631434605833),
 ('рейберн', 0.013433775577766705),
 ('мистический', 0.013047284540497209),
 ('бесшумный', 0.012942671472308018),
 ('пейзаж', 0.012870093606007965),
 ('квеста', 0.012687427912341407),
 ('загадка', 0.012687365806859625),
 ('николай', 0.011928777681208647),
 ('роза', 0.011834319526627219)]

Как видно, в списках очень много каких-то случайных существительных. Я решила, что будет намного показательнее, если в списках останутся только прилагательные. Это должно увеличить точность оценки, даже несмотря на то, что некоторые "полезные" существительные и глаголы уберутся.

#### Функция для отбора прилагательных

In [41]:
def get_adj(freq):
    result = Counter()
    for word in dict(freq).keys():
        if morph.parse(word)[0].tag.POS == 'ADJF':
            result[word] = freq[word]
    return result

In [42]:
adj_neg = get_adj(neg_freq)
adj_pos = get_adj(pos_freq)

In [43]:
adj_neg.most_common(15)

[('неприятный', 0.017958297260874202),
 ('обычный', 0.00948041743980214),
 ('откровенный', 0.00875171135994731),
 ('русский', 0.008327136337893296),
 ('худой', 0.007745603652079546),
 ('крутой', 0.007576847671636959),
 ('слабый', 0.007369154145425918),
 ('отвратительный', 0.006828120507815455),
 ('эмоциональный', 0.006717694823274509),
 ('голливудский', 0.005462976009017432),
 ('малопонятный', 0.0047169811320754715),
 ('бредовый', 0.0047169811320754715),
 ('неубедительный', 0.0045662100456621),
 ('типичный', 0.004538405400474366),
 ('грязный', 0.004527713582857983)]

In [56]:
adj_pos.most_common(15)

[('мистический', 0.013047284540497209),
 ('бесшумный', 0.012942671472308018),
 ('детективный', 0.011606503497547806),
 ('местный', 0.010857005095975591),
 ('интригующий', 0.009847310183448105),
 ('неизвестный', 0.006014741419145664),
 ('штормовой', 0.005494505494505495),
 ('удивительный', 0.005402930402930403),
 ('одинокий', 0.005402930402930403),
 ('грустный', 0.005402930402930403),
 ('истинный', 0.005372811820186004),
 ('душевный', 0.00510752688172043),
 ('морской', 0.00510752688172043),
 ('невероятный', 0.005106199107495399),
 ('документальный', 0.004943170683911424)]

Получились относительно нормальные списки с наличием подходящих оценочных слов.

#### Функция собирает тестовую выборку
Скачивается не менее 10 отзывов с их оценкой.

In [8]:
# test_link = 'https://irecommend.ru/category/filmy?page=499'

def get_test_texts(n):
    text_dict = {}
    b_count_all = 0
    g_count_all = 0

    soup = get_soup('/category/filmy?page=499')
    all_books = soup.find_all('a', {'class': 'read-all-reviews-link-bottom read-all-reviews-link'})

    for book in all_books:
        b_count = 0
        g_count = 0
        if b_count_all > n and g_count_all > n:
            break
        sleep(10)
        soup = get_soup(book.get('href'))
        all_reviews = soup.find_all('a', {'class': 'more'})
        
        for review in all_reviews:
            sleep(10)
            soup = get_soup(review.get('href'))
            recommend = soup.find('span', {'class': 'verdict'})
            
            if recommend.text == 'не рекомендует' and b_count <= 2 and b_count_all <= n:
                text = soup.find('div', {'class': 'description hasinlineimage'})
                text_dict[text.text] = recommend.text
                b_count += 1
                b_count_all += b_count
                
            elif g_count <= 2 and g_count_all <= n:
                text = soup.find('div', {'class': 'description hasinlineimage'})
                text_dict[text.text] = recommend.text
                g_count += 1
                g_count_all += g_count
                
            elif g_count > 2 and b_count > 2:
                break
                
    return text_dict

In [11]:
test_dict = get_test_texts(10)

#### Функция, предсказывающая оценку
Итоговые положительные и отрицательные очки делятся на суммы всех значений в соответствующих словарях, чтобы получить процентные соотношения. Нельзя сравнивать очки, потому что суммы значений в разных оценках могут отличаться.

In [44]:
def predict_verdict(text, adj_pos, adj_neg):
    pos_score = 0
    neg_score = 0
    words = nltk.word_tokenize(text.lower())
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    for lemma in lemmas:
        if lemma in adj_pos.keys():
            pos_score += adj_pos[lemma]
        elif lemma in adj_neg.keys():
            neg_score += adj_neg[lemma]
    n_score = neg_score/sum(adj_neg.values())
    p_score = pos_score/sum(adj_pos.values())
    return p_score, n_score

Посмотрим на совпадения предсказаний и правильных ответов. Так получилось, что в 3 текстах вообще не встретились слова из собранных словарей, поэтому предсказать оценку невозможно. 

In [61]:
for text in test_dict.keys():
    p_score, n_score = predict_verdict(text, adj_pos, adj_neg)
    if p_score > n_score:
        print('predict: positive')
    elif p_score == n_score:
        print('predict: can not predict')
    else:
        print('predict: negative')
    print('answer:', test_dict[text], '\n')

predict: positive
answer: рекомендует 

predict: positive
answer: не рекомендует 

predict: negative
answer: не рекомендует 

predict: negative
answer: не рекомендует 

predict: can not predict
answer: не рекомендует 

predict: positive
answer: рекомендует 

predict: can not predict
answer: рекомендует 

predict: positive
answer: рекомендует 

predict: positive
answer: не рекомендует 

predict: positive
answer: рекомендует 

predict: can not predict
answer: не рекомендует 

predict: negative
answer: не рекомендует 



Посчитаем accuracy. Получилось 75%. Но это не совсем правда, потому что у нас был "третий класс" отзывов, оценку которых не получилось определить. Такие отзывы определялись в отрицательные.

In [53]:
# 1 - pos, 0 - neg
predict = []
gold = []
for text in test_dict.keys():
    p_score, n_score = predict_verdict(text, adj_pos, adj_neg)
    if p_score > n_score:
        predict.append(1)
    else:
        predict.append(0)
    if test_dict[text] == 'рекомендует':
        gold.append(1)
    else:
        gold.append(0)

print("Accuracy: %.4f" % accuracy_score(predict, gold))

Accuracy: 0.7500


Улучшить показатели можно было бы с помощью увеличения выборки. Я взяла минимум (30 слов на каждый тип), потому что без долгих sleep'ов приходилось бы мучаться с капчами, а с ними работает довольно долго. С большей выборкой было бы большее разнообразие фильмов и оценочные слова в итоге оказались бы в топе по частотности.

Еще было бы полезно сбаланировать выборку. В моей программе был установлен только нижний порог по длине, но можно было бы ограничить ее и сверху. В целом, было бы неплохо уравнять тексты как можно лучше.